# Initial Steps

## Install Dependencies

In [7]:
!pip install --upgrade pip
# !pip install google-cloud-aiplatform

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.0 MB/s eta 0:00:00m eta 0:00:01:01:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1
    Uninstalling pip-24.1:
      Successfully uninstalled pip-24.1


## Setup GCP and VertexAI

In [20]:
import json
with open("../resources/config.json") as f:
    config = json.load(f)
    

In [23]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

In [27]:
# Path to your service account key file
key_path = "../resources/keys.json" #Path to the json key associated with your service account from google cloud

In [28]:
# Create credentials object

credentials = Credentials.from_service_account_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/cloud-platform'])

if credentials.expired:
    credentials.refresh(Request())

In [33]:
# Connect to vertexAI
import vertexai

PROJECT_ID = config['gcp']['project_id']
REGION = config['gcp']['region']

# initialize vertex
vertexai.init(project = PROJECT_ID, location = REGION, credentials = credentials)

## Get start with text embeddings

In [35]:
from vertexai.language_models import TextEmbeddingModel

In [39]:
# load the model

embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko")

In [40]:
# Generate word embeddigs
# The returned object is a list with a single TextEmbedding object.
# The TextEmbedding.values field stores the embeddings in a Python list.

embedding = embedding_model.get_embeddings(
    ["life"]
)

In [44]:
vector = embedding[0].values

print(f"Length of vector = {len(vector)}")
print(vector)

Length of vector = 768
[0.013755870051681995, -0.03912805765867233, -0.008546194061636925, -0.02595570497214794, 0.01972811669111252, -0.014042137190699577, -0.006056010257452726, -0.010783486068248749, -0.0036132442764937878, -0.00030527994385920465, -0.006657462567090988, -0.00985203217715025, 0.01936277560889721, -0.05349944159388542, 0.0060171219520270824, -0.03781122341752052, 0.026209799572825432, 0.03440653905272484, 0.0026221326552331448, 0.004484028089791536, -0.04308246448636055, 0.0072737219743430614, -0.0010411994298920035, 0.006253989413380623, 0.006622487213462591, 0.0009960986208170652, 0.03888087719678879, -0.09485860913991928, -0.012705202214419842, 0.05182832106947899, -0.0684097409248352, 0.0195312462747097, -0.07294485718011856, 0.029158100485801697, 0.010619454085826874, -0.021925820037722588, 0.04528914764523506, 0.0004353086987975985, -0.010649383999407291, -0.01780308224260807, -0.0019414034904912114, -0.02967706322669983, -0.06325230002403259, -0.02656076289713